# Local-Global-Trend(LGT) Example

In [1]:
import pandas as pd
import numpy as np
from orbit.lgt import LGT
from orbit.utils.plot import plot_predicted_data
from orbit.utils.plot import plot_predicted_components

## Data

In [2]:
DATA_FILE = "./data/iclaims.example.csv"

In [3]:
raw_df = pd.read_csv(DATA_FILE, parse_dates=['week'])

In [4]:
raw_df.dtypes

week              datetime64[ns]
claims                     int64
trend.unemploy           float64
trend.filling            float64
trend.job                float64
dtype: object

In [5]:
raw_df.head(5)

,week,claims,trend.unemploy,trend.filling,trend.job
0,2010-01-03,651215,1.183973,0.720140,1.119669
1,2010-01-10,825891,1.183973,0.814896,1.178599
2,2010-01-17,659173,1.203382,0.739091,1.119669
3,2010-01-24,507651,1.164564,0.814896,1.107883
4,2010-01-31,538617,1.086926,0.776993,1.072525


### Train / Test Split

In [6]:
df=raw_df.copy()
test_size=52
train_df=df[:-test_size]
test_df=df[-test_size:]

In [7]:
train_df.head(5)

,week,claims,trend.unemploy,trend.filling,trend.job
0,2010-01-03,651215,1.183973,0.720140,1.119669
1,2010-01-10,825891,1.183973,0.814896,1.178599
2,2010-01-17,659173,1.203382,0.739091,1.119669
3,2010-01-24,507651,1.164564,0.814896,1.107883
4,2010-01-31,538617,1.086926,0.776993,1.072525


## MAP Fit and Predict

In [8]:
lgt_map=LGT(
    response_col="claims",
    date_col="week",
    seasonality=52,
    seed=8888,
    inference_engine='pyro',
    predict_method='map',
    auto_scale=False,
    is_multiplicative=True
)

In [ ]:
lgt_map.fit(df=train_df)

step    0 loss = 5864.7
step   10 loss = -122.68
step   20 loss = -440.17
step   30 loss = -510.49
step   40 loss = -528.49
step   50 loss = -527.4
step   60 loss = -538.01
step   70 loss = -531.82
step   80 loss = -543.12
step   90 loss = -531.36


In [ ]:
predicted_df = lgt_map.predict(df=test_df)

In [ ]:
plot_predicted_data(training_actual_df=train_df, predicted_df=predicted_df, 
                    date_col=lgt_map.date_col, actual_col=lgt_map.response_col, pred_col='prediction', 
                    test_actual_df=test_df)

In [ ]:
test_df.head(5)

## VI Fit and Predict

Pyro only support Variational Inference(SVI) for full sampling prediction.

In [ ]:
lgt_svi = LGT(
    response_col='claims',
    date_col='week',
    seasonality=52,
    num_warmup=4000, 
    num_sample=500,
    seed=8888,
    inference_engine='pyro',
    sample_method='vi',
    predict_method='full',
    n_bootstrap_draws=500
)

In [ ]:
lgt_svi.fit(df=train_df)

In [ ]:
predicted_df = lgt_svi.predict(df=test_df)

In [ ]:
predicted_df.tail(5)

In [ ]:
plot_predicted_data(training_actual_df=train_df, predicted_df=predicted_df, 
                    date_col=lgt_svi.date_col, actual_col=lgt_svi.response_col, pred_col=50, 
                    pred_quantiles_col=[5,95], test_actual_df=test_df)

## Mean Fit

In [ ]:
lgt_mean=LGT(
    response_col='claims',
    date_col='week',
    seasonality=52,
    seed=8888,
    inference_engine='pyro',
    predict_method='mean'
)

In [ ]:
lgt_mean.fit(df=train_df)

In [ ]:
predicted_df = lgt_mean.predict(df=test_df)

In [ ]:
predicted_df.head(5)

In [ ]:
plot_predicted_data(training_actual_df=train_df, predicted_df=predicted_df, 
                    date_col=lgt_mean.date_col, actual_col=lgt_mean.response_col, pred_col='prediction', 
                    test_actual_df=test_df)

## Fit with Regressors [WIP]

In [ ]:
# lgt_reg_map=LGT(
#     response_col='claims',
#     regressor_col=['trend.unemploy', 'trend.filling'],
#     date_col='week',
#     seasonality=52,
#     seed=8888,
#     inference_engine='pyro',
#     predict_method='map'
# )

In [ ]:
# lgt_reg_map.fit(df=train_df)

### Decompose Prediction

We can use `decompose=True` to decompose prediction into `trend`, `seasonality` and `regression` components. This command only works with point estimate.

In [ ]:
# # make prediction of past and future
# # TODO: Got ERROR in this block
# predicted_df = lgt_reg_map.predict(df=df, decompose=True)
# predicted_df.head(5)

In [ ]:
# plot_predicted_components(predicted_df=predicted_df, date_col='week')